In [ ]:
#libraries
from scipy.signal import butter, filtfilt, spectrogram
import scipy.stats as ss
import pandas as pd
import numpy as np

In [ ]:
%matplotlib qt

In [ ]:
data = pd.read_pickle("all_subjects.pkl")

In [ ]:
signal = data['ch0'][0][0]
signal.shape

#### EEG rhythms:
$\alpha = 8-13 \text{Hz}, \ \ \beta = 13-30 \text{Hz},  \ \ \delta = 0.5-4 \text{Hz},  \ \ \theta = 4-8 \text{Hz},  \ \ \gamma = 30-60 \text{Hz}$ 

In [ ]:
def bandpass_filter(signal, low_cutoff, high_cutoff, sample_rate, order=5):
    nyquist_freq = 0.5 * sample_rate
    low = low_cutoff / nyquist_freq
    high = high_cutoff / nyquist_freq
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

In [ ]:
fs = 500
N = len(signal) 
time = np.arange(N) / fs

In [ ]:
alpha_rhythm = bandpass_filter(signal, 8, 13, fs)
beta_rhythm = bandpass_filter(signal, 13, 30, fs)
delta_rhythm = bandpass_filter(signal, 0.5, 4, fs)
theta_rhythm = bandpass_filter(signal, 4, 8, fs)
gamma_rhythm = bandpass_filter(signal, 30, 60, fs)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(1)
plt.plot(time, gamma_rhythm, label='Gamma')
plt.plot(time, alpha_rhythm, label='Alpha')
plt.legend()

In [ ]:
def plot_spect(data, N, fs):
    freq_data = 2 * (np.fft.fft(data) / N)[:N//2+1]
    freqs = np.fft.fftfreq(N, 1/fs)[:N//2+1]
    plt.plot(freqs, np.abs(freq_data))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.title('One-Sided Spectrum')
    plt.xlim([0, 45])
    plt.show()

In [ ]:
plot_spect(signal, N, fs)
plot_spect(alpha_rhythm, N, fs)

#### Features

##### Mean

In [ ]:
mean = np.mean(signal)
mean

##### Variance

In [ ]:
var = np.var(alpha_rhythm[0:10*fs])
var

##### STD

In [ ]:
std = np.std(alpha_rhythm[0:10*fs])
std

##### Skewness

In [ ]:
skew = ss.skew(alpha_rhythm[0:10*fs])
skew

##### Kurtosis

In [ ]:
kurt = ss.kurtosis(alpha_rhythm[0:10*fs])
kurt

#### Comparing basic features between subjects

In [ ]:
def get_subject_eeg(index, chanel):
    signal = data['ch' + str(chanel)][index][0]
    return signal

In [ ]:
window = 5 # epoch length in seconds
overlap = 2.5 # length of overlap in seconds
window_length, overlap_length = int(window*fs), int(overlap*fs)

sub1, sub2 = 0, 41
signal0 = get_subject_eeg(sub1, 0)
signal1 = get_subject_eeg(sub2, 0)
print("MMSE score of first subject is {} and of second {}".format(data['MMSE'][sub1], data['MMSE'][sub2]))


In [ ]:
plt.plot(bandpass_filter(signal0, 4, 8, fs)[100*fs:110*fs])
plt.plot(bandpass_filter(signal1, 4, 8, fs)[100*fs:110*fs])

### Comparing amplitudes of alpha and theta rhythms

In [73]:
alpha = np.empty((88, 1))
theta = np.empty((88, 1))
mmse = np.empty((88, 1))
for i in range(88):
    signal = get_subject_eeg(i, 0)
    alpha_signal = bandpass_filter(signal, 13, 30, fs)
    theta_signal = bandpass_filter(signal, 4, 8, fs)
    alpha[i] = np.sum(np.square(alpha_signal))/len(signal)
    theta[i] = np.sum(np.square(theta_signal))/len(signal)
    mmse[i] = data['MMSE'][i]

plt.figure(1)
plt.scatter(mmse, alpha)
plt.xlabel('mmse score')
plt.ylabel('power')
plt.title('power in alpha spectrum')
plt.figure(2)
plt.scatter(mmse, theta)
plt.xlabel('mmse score')
plt.ylabel('power')
plt.title('power in theta spectrum')


Text(0.5, 1.0, 'power in theta spectrum')